In [ ]:
# Import the packages 
import math
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pandas.plotting import table
from itertools import permutations 
# import dataframe_image as dfi
import matplotlib.pyplot as plt
from io import BytesIO
from PIL import Image
from prettytable import PrettyTable
from tabulate import tabulate
#from tabulate import table

In [ ]:
def get_permutations(numbers):
    return list(permutations(numbers))

def convert_to_0s(photon):
    digit_to_0s = [(i-1)*"0" for i in photon]
    return digit_to_0s
    
def AddPolarization(digit_to_0s):
    result = ['']
    prev_non_empty = None
    
    for i, elem in enumerate(digit_to_0s):
        if i == 0:
            prev_non_empty = elem
            result.append(elem)
            continue
        
        if elem.count('0') > prev_non_empty.count('0'):
            result[-1] = 'H' + result[-1]
        elif elem.count('0') < prev_non_empty.count('0'):
            result[-1] = 'V' + result[-1]
        
        result.append(elem)
        prev_non_empty = elem
    
    if len(result[0]) < len(result[-1]):
        result[-1] = 'V' + result[-1][1:] + '0'
    elif len(result[0]) > len(result[-1]):
        result[-1] = 'H' + result[-1][1:] + '0'
    elif len(digit_to_0s) == 2:
        result[-1] = 'H'
    
    if len(digit_to_0s) > 1:
        last_digit_zeros = digit_to_0s[-1].count('0')
        second_last_digit_zeros = digit_to_0s[-2].count('0')
        if last_digit_zeros == 0:
            if second_last_digit_zeros > 0:
                result[-1] = result[-1][:-1] + 'H'
            else:
                result[-1] += 'H'
        elif result[-1][-1] == '1':
            if last_digit_zeros > second_last_digit_zeros:
                result[-1] = result[-1][:-1] + 'V'
            elif second_last_digit_zeros > last_digit_zeros:
                result[-1] = result[-1][:-1] + 'H'
            else:
                result[-1] = result[-1][:-1] + 'V' if result[-2][-1] == 'H' else result[-1][:-1] + 'H'
    
    return ''.join(result)      # Join the result list into a single string

def convert_to_one_string(numList):         # [1,2,3]
      s = map(str, numList)   # ['1','2','3']
      s = ''.join(s)          # '123'
      s = f'{s}'
      #s = int(s)              # 123
      return s

def flip_polarizations(polarization_string):
    output = {polarization_string}
    for i in range(1, len(polarization_string) + 1): # loop through all possible flip counts
        for j in range(len(polarization_string)): # loop through each polarization
            if polarization_string[j] != '0':
                output |= {polarization_string[:j] + '0' + polarization_string[j+1:]}
                for k in range(j+1, len(polarization_string)):
                    if polarization_string[k] != '0':
                        new_string = polarization_string[:j] + '0' + polarization_string[j+1:k] + '0' + polarization_string[k+1:]
                        output.add(new_string)
        if i == len(polarization_string): # stop flipping if there are no more polarizations to flip
            break
        new_output = set()
        for string in output:
            for j in range(len(polarization_string)):
                if string[j] != '0':
                    for k in range(j+1, len(polarization_string)):
                        if string[k] != '0':
                            new_string = string[:j] + '0' + string[j+1:k] + '0' + string[k+1:]
                            new_output.add(new_string)
        output |= new_output

    output.discard(polarization_string) # remove the original string from the output
    return sorted(output) # return a sorted list of polarization strings.


photons_2 = get_permutations([1,2]) 
photons_3 = get_permutations([1,2,4])
photons_4 = get_permutations([1, 2, 4, 7]) 
photons_5 = get_permutations([1, 2, 4, 7, 12]) 

photons = [photons_2, photons_3, photons_4, photons_5]

dictionaries = []

for i, photon_set in enumerate(photons, 2):
    dictionary = {"Sent Digits": photon_set, "Encoded Words": []}
    flipped_list = []

    for j, sub_list in enumerate(photon_set):
        elements = convert_to_one_string(AddPolarization(convert_to_0s(sub_list)))
        dictionary["Encoded Words"].append(elements)
        flipped = flip_polarizations(elements)
        flipped_list.append([''.join(f) for f in flipped])
    dictionary[f"Received Loss Errors"] = flipped_list

    dictionaries.append(dictionary)


# for i, dictionary in enumerate(dictionaries):
#     print(f"Dictionary of {i+2} photons: {dictionary}")


# Define table headers
headers = ["Sent Digits", "Encoded Words", "Received Loss Errors"] #"Encoded Words with Loss Errors"

# Create table object
table = PrettyTable(headers)

# # Set the maximum width for each column
# table.max_width["Sent Digits"] = 15
# table.max_width["Encoded Words"] = 30
# table.max_width["Received Loss Errors"] = 30

# Add rows to table
for d in dictionaries:
    # Get photons and sums values from dictionary
    photons = d["Sent Digits"]
    encoded_words = d["Encoded Words"]
    loss_errors = d["Received Loss Errors"]
    #encoded_words_with_loss_errors = d["Encoded Words with Loss Errors"]
    
    # Add rows to the table
    for i in range(len(photons)):
        row = [photons[i], encoded_words[i], loss_errors[i]] # # encoded_words_with_loss_errors[i]
        table.add_row(row)


# def check_codeword(codeword, table):
#     for row in table._rows:
#         sent_digit = row[0]
#         encoded_word = row[1]
#         received_loss_error = row[2]

#         if codeword == received_loss_error:
#             print(f"Your received code word '{codeword}' is incorrect, it should be the coded word '{encoded_word}' from the 'Encoded Words', which corresponds to the sent digit {sent_digit} from the dictionary 'Sent Digits'.")
#             return

#     print("The provided code word is not found in the 'Received Loss Errors' column in the table.")

# #print(table)

In [ ]:
# def check_codeword(codeword, table):
#     if all(c == '0' for c in codeword):
#         print("I cannot determine the correct code word. Please check the table.")
#         return

#     for row in table._rows:
#         sent_digit = row[0]
#         encoded_word = row[1]
#         received_loss_error = row[2]

#         if codeword == encoded_word:
#             print(f"Your received code word '{codeword}' is correct, which corresponds to the sent digit {sent_digit} from the dictionary 'Sent Digits'.")
#             return
    
#     for row in table._rows:
#         sent_digits = row[0]
#         encoded_word = row[1]
#         received_loss_errors = row[2]

#         if codeword in received_loss_errors:
#            print(f"Matching row found:\nYour received code word is in the Received loss error, which belongs to one of the possibilities: '{received_loss_errors}'\nPossible Sent Encoded word: {encoded_word}\nCorresponding Sent digits: {sent_digits}\n")   
    
#     print("The provided code word is not found in either 'Encoded Words' or 'Received Loss Errors' columns in the table.")

In [ ]:
def check_codeword(codeword, table):
    if all(c == '0' for c in codeword):
        print("I cannot determine the correct code word. Please check the table.")
        return

    found_match = False

    for row in table._rows:
        sent_digit = row[0]
        encoded_word = row[1]
        received_loss_error = row[2]

        if codeword == encoded_word:
            print(f"Your received code word '{codeword}' is correct, which corresponds to the sent digit {sent_digit} from the dictionary 'Sent Digits'.")
            return
    
    for row in table._rows:
        sent_digits = row[0]
        encoded_word = row[1]
        received_loss_errors = row[2]

        if codeword in received_loss_errors:
            found_match = True
            print(f"Matching row found:\nYour received code word is in the Received loss error, which belongs to one of the possibilities: '{received_loss_errors}'\nPossible Sent Encoded word: {encoded_word}\nCorresponding Sent digits: {sent_digits}\n")   

    if not found_match:
        print("The provided code word is not found in either 'Encoded Words' or 'Received Loss Errors' columns in the table.")


In [ ]:
print(table[0:2]) # 2! = 2

+-------------+---------------+-----------------------+
| Sent Digits | Encoded Words |  Received Loss Errors |
+-------------+---------------+-----------------------+
|    (1, 2)   |      HV0      | ['000', '0V0', 'H00'] |
|    (2, 1)   |      V0H      | ['000', '00H', 'V00'] |
+-------------+---------------+-----------------------+


In [ ]:
print(table[2:8]) # 3! = 6

+-------------+---------------+-------------------------------------------------------------------------------+
| Sent Digits | Encoded Words |                              Received Loss Errors                             |
+-------------+---------------+-------------------------------------------------------------------------------+
|  (1, 2, 4)  |    HH0V000    | ['0000000', '000V000', '0H00000', '0H0V000', 'H000000', 'H00V000', 'HH00000'] |
|  (1, 4, 2)  |    HV000V0    | ['0000000', '00000V0', '0V00000', '0V000V0', 'H000000', 'H0000V0', 'HV00000'] |
|  (2, 1, 4)  |    V0HV000    | ['0000000', '000V000', '00H0000', '00HV000', 'V000000', 'V00V000', 'V0H0000'] |
|  (2, 4, 1)  |    H0V000H    | ['0000000', '000000H', '00V0000', '00V000H', 'H000000', 'H00000H', 'H0V0000'] |
|  (4, 1, 2)  |    V000HV0    | ['0000000', '00000V0', '0000H00', '0000HV0', 'V000000', 'V0000V0', 'V000H00'] |
|  (4, 2, 1)  |    V000V0H    | ['0000000', '000000H', '0000V00', '0000V0H', 'V000000', 'V00000H', 'V000

In [ ]:
print(table[8:32]) # 4! = 24

+--------------+----------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| Sent Digits  | Encoded Words  |                                                                                                                              Received Loss Errors                                                                                                                              |
+--------------+----------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| (1, 2, 4, 7) | HH0H000V000000 | ['00000000000000', '0000000V000000', '000H000

In [ ]:
check_codeword('0H0V000', table)

Matching row found:
Your received code word is in the Received loss error, which belongs to one of the possibilities: '['0000000', '000V000', '0H00000', '0H0V000', 'H000000', 'H00V000', 'HH00000']'
Possible Sent Encoded word: HH0V000
Corresponding Sent digits: (1, 2, 4)



In [ ]:
check_codeword('0000000V0000V0', table)

Matching row found:
Your received code word is in the Received loss error, which belongs to one of the possibilities: '['00000000000000', '000000000000V0', '00000000000H00', '00000000000HV0', '0000000V000000', '0000000V0000V0', '0000000V000H00', '0000000V000HV0', 'V0000000000000', 'V00000000000V0', 'V0000000000H00', 'V0000000000HV0', 'V000000V000000', 'V000000V0000V0', 'V000000V000H00']'
Possible Sent Encoded word: V000000V000HV0
Corresponding Sent digits: (7, 4, 1, 2)



In [ ]:
check_codeword('V0H', table)

Your received code word 'V0H' is correct, which corresponds to the sent digit (2, 1) from the dictionary 'Sent Digits'.


In [ ]:
check_codeword('V000000V000V0H', table)

Your received code word 'V000000V000V0H' is correct, which corresponds to the sent digit (7, 4, 2, 1) from the dictionary 'Sent Digits'.


In [ ]:
check_codeword('00000000', table)

I cannot determine the correct code word. Please check the table.


In [ ]:
check_codeword('0H0252452', table)

The provided code word is not found in either 'Encoded Words' or 'Received Loss Errors' columns in the table.


In [ ]:
check_codeword('V000000', table)

Matching row found:
Your received code word is in the Received loss error, which belongs to one of the possibilities: '['0000000', '000V000', '00H0000', '00HV000', 'V000000', 'V00V000', 'V0H0000']'
Possible Sent Encoded word: V0HV000
Corresponding Sent digits: (2, 1, 4)

Matching row found:
Your received code word is in the Received loss error, which belongs to one of the possibilities: '['0000000', '00000V0', '0000H00', '0000HV0', 'V000000', 'V0000V0', 'V000H00']'
Possible Sent Encoded word: V000HV0
Corresponding Sent digits: (4, 1, 2)

Matching row found:
Your received code word is in the Received loss error, which belongs to one of the possibilities: '['0000000', '000000H', '0000V00', '0000V0H', 'V000000', 'V00000H', 'V000V00']'
Possible Sent Encoded word: V000V0H
Corresponding Sent digits: (4, 2, 1)

